In [1]:
import os
import numpy as np
from anndata import AnnData
from anndata._io import read_zarr
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
import napari
from napari_ome_zarr._reader import transform
from napari.types import LayerDataTuple

from ngff_tables_prototype.reader import load_points_to_anndata, _anndata_to_napari_points, load_to_napari_viewer

In [2]:
# for the conversion from tracking data matrix to napari tracks format

def check_div(tracks_matrix, n):
    next = 0
    for i in range(len(tracks_matrix[n])):
            if tracks_matrix[n][i] == 1:
                next += 1
    return next

def track_obj(tracks_matrix, m, points_coords, tracks, tid):
    if check_div(tracks_matrix, m) == 1:
        tracks = np.append(tracks, np.insert(points_coords[m], 0, tid))
        for i in range(len(tracks_matrix[m])):
            if tracks_matrix[m][i] == 1:
                tracks_matrix[m][i] = 2
                tracks = track_obj(tracks_matrix, i, points_coords, tracks, tid)
        return tracks
    else:
        return tracks
                
def _anndata_to_napari_tracks(anndata_obj: AnnData) -> LayerDataTuple:
    points_coords = anndata_obj.X
    tracks_matrix = anndata_obj.obsp["tracking"]
    tracks = np.empty((0, 5))
    
    tid = 0
    for i in range(len(tracks_matrix)):
        for j in range(len(tracks_matrix[i])):
            if tracks_matrix[i][j] == 1:
                tracks = np.append(tracks, np.insert(points_coords[i], 0, tid))
                tracks = track_obj(tracks_matrix, j, points_coords, tracks, tid)
                tid += 1

    t = tracks.reshape([tracks.size // 5, 5])

    return t, "", 'tracks'

In [3]:
# copied from https://github.com/kevinyamauchi/ome-ngff-tables-prototype/blob/main/src/ngff_tables_prototype/reader.py

def load_to_napari_layer_bdz(file_path: str, points_group: str):
    ome_zarr = parse_url(file_path)
    reader = Reader(ome_zarr)
    reader_func = transform(reader())
    layer_data = reader_func(file_path)

    # load points points
    anndata_obj = load_points_to_anndata(file_path, points_group)
    layer_data.append(_anndata_to_napari_points(anndata_obj))

    # load tracking info. (added)
    layer_data.append(_anndata_to_napari_tracks(anndata_obj))
    
    return layer_data

def load_to_napari_viewer_bdz(file_path: str, points_group: str) -> napari.Viewer:
    layer_data = load_to_napari_layer_bdz(file_path, points_group)

    viewer = napari.Viewer()

    for layer in layer_data:
        viewer._add_layer_from_data(*layer)
    return viewer

In [4]:
viewer = load_to_napari_viewer_bdz(file_path='wt-N2-081015-01.ome.zarr', points_group='dyn')
napari.run()

version mismatch: detected:FormatV02, requested:FormatV04
version mismatch: detected:FormatV04, requested:FormatV02
version mismatch: detected:FormatV02, requested:FormatV04
version mismatch: detected:FormatV04, requested:FormatV02
/usr/local/lib/python3.9/site-packages/napari/_qt/qt_resources/_icons.py:350: UserWarning: Unable to save qt-resources: [Errno 13] Permission denied: '/usr/local/lib/python3.9/site-packages/napari/_qt/qt_resources/_qt_resources_PyQt5_5_15_2_d12ed21803a677a6c4be7449d5c830ba.py'
  warnings.warn(msg)
